# Retrieval Hands On Lab

## Objectives
By the end of this lab, participants will:

1. Understand how to parse PDFs inside Snowflake
2. Understand how to create vector representations of text data and load it into Snowflake tables
3. Perform similarity search against embeddings in Snowflake
4. Use Snowflake Cortex Search for retrieval and understand the benefits compared to simple similarity search

# Part 1: Setup
In this section, we will:

1. Create some snowflake objects to store our data in
2. Upload a PDF of Cincinnati Parks' 3 year development plan into a stage
3. Parse the PDF into usable text and load the results into a Snowflake table

## Inspect the PDF

Download the Cincinnati Parks 3 year plan by clicking on the data directory, then clicking the '...' next to the PDF and choosing 'Download'.

## Add python packages

Click on 'Packages' at the top of the notebook and add the following packages:

- snowflake.core
- snowflake-snowpark-python
- snowflake-ml-python

In [ ]:
USE GAIG_LAB.PUBLIC;

-- Create a stage to store our PDF
CREATE OR REPLACE STAGE docs ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root

session = get_active_session()
pdf_path = "./data/cincinnati-parks-3-year-plan.pdf"

session.file.put(
    pdf_path,
    "@docs",
    auto_compress=False,
    overwrite=True
)

# Force a refresh on the stage so the doc is accessible
session.sql("ALTER STAGE docs REFRESH").collect()

In [ ]:
-- Verify PDF was properly uploaded
LIST @docs;

In [ ]:
-- This table will store the text from the parsed PDF
CREATE OR REPLACE TABLE PARSED_PDFS ( 
    RELATIVE_PATH VARCHAR,
    SIZE NUMBER(38,0),
    FILE_URL VARCHAR,
    PARSED_DATA VARCHAR);

In [ ]:
-- We use Snowflake Cortex's PARSE_DOCUMENT function to extract the text from the pdf and save it to a column
INSERT INTO PARSED_PDFS (relative_path, size, file_url, parsed_data)
SELECT 
        relative_path,
        size,
        file_url,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@docs', relative_path, { 'mode': 'OCR' }):content AS parsed_data
    FROM directory(@docs);

In [ ]:
-- Verify the data was successfully parsed
select * from PARSED_PDFS;

## Part 2 - Generate Embeddings

In this section, we will:

1. Explore various strategies for chunking the text data
2. Generate embeddings for our text chunks
3. Load the results into a Snowflake table using the `VECTOR` datatype

### Chunking Strategies

In this section, we'll explore various chunking strategies. The right strategy will ultimately depend on the data and use case at hand. In our example, the PDF is cleanly delineated into paragraphs, so a simple regex based chunker is ideal.

1. Snowflake Recursive Text Splitter
2. Semantic Chunking
3. Simple Chunking

In [ ]:
-- Test out Snowflake's built in recursive text chunker
SELECT
  f.value::string AS chunk
FROM
  PARSED_PDFS,
  LATERAL FLATTEN(
    INPUT => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
      PARSED_DATA, -- column to split
      'none', -- format ('none' or 'markdown')
      1000, -- chunk size
      100 -- overlap
    )
  ) f;

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root

session = get_active_session()

parsed_data_df = session.table('parsed_pdfs')
parsed_text = parsed_data_df.collect()[0]

# Create chunks for each park and its project plan
def is_title(line):
    stripped = line.strip()
    return (
        bool(stripped) and 
        stripped == stripped.upper() and 
        any(c.isalpha() for c in stripped)
    )

def chunk_by_project(parsed_text):
    lines = parsed_text["PARSED_DATA"].splitlines()
    chunks = []
    current_title = None
    current_desc_lines = []
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if is_title(line):
            # Check if the next line is also a title (part of same heading)
            title_lines = [line]
            while i + 1 < len(lines) and is_title(lines[i + 1].strip()):
                i += 1
                title_lines.append(lines[i].strip())
            # If we already have a title and description, save that chunk
            if current_title:
                chunk = f"{current_title}\n{' '.join(current_desc_lines).strip()}"
                chunks.append({
                    "relative_path": parsed_text["RELATIVE_PATH"],
                    "size": parsed_text["SIZE"],
                    "file_url": parsed_text["FILE_URL"],
                    "chunk": chunk
                })
            # Start a new chunk
            current_title = ' '.join(title_lines)
            current_desc_lines = []
        else:
            current_desc_lines.append(line)
        i += 1

    # Add the last chunk
    if current_title and current_desc_lines:
        chunk = f"{current_title}\n{' '.join(current_desc_lines).strip()}"
        chunks.append({
            "relative_path": parsed_text["RELATIVE_PATH"],
            "size": parsed_text["SIZE"],
            "file_url": parsed_text["FILE_URL"],
            "chunk": chunk
        })

    return chunks

chunks = chunk_by_project(parsed_text)

In [ ]:
# View chunks
for idx, chunk in enumerate(chunks):
    print(f'chunk {idx}:', chunk['chunk'])

In [ ]:
from snowflake.cortex import embed_text_768

# Create embeddings for each chunk
model = 'snowflake-arctic-embed-m-v1.5'
for chunk in chunks:
    chunk['embedding'] = embed_text_768(model, chunk['chunk'], session)
    

In [ ]:
from snowflake.snowpark.types import VectorType, DoubleType

# Save off chunks and embeddings into new table
df = session.create_dataframe(chunks)
df = df.with_column('embedding', df.col('embedding').cast(VectorType(float, 768)))
df.write.save_as_table("DOCS_CHUNKS_TABLE")

In [ ]:
-- Validate table
select chunk, embedding from DOCS_CHUNKS_TABLE;

## Part 3: Test out different search methods

In this section, we will:

1. Perform a standard cosine similarity search
2. Create a Cortex Search Service
3. Perform a search against the Cortex Search Service

In [ ]:
-- Ault Park Trail
SELECT VECTOR_COSINE_SIMILARITY(
            docs_chunks_table.embedding,
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m-v1.5', 'When will the Ault Park trail plan complete?')
       ) as similarity,
       chunk
FROM docs_chunks_table
ORDER BY similarity desc
LIMIT 10
;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE parks_search_service
  ON CHUNK
  WAREHOUSE = compute_wh
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5'
  AS (
    SELECT
        CHUNK,
    FROM docs_chunks_table
);

In [ ]:
# Quick test of the search service
import json
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root

session = get_active_session()

root = Root(session)
parks_search_service = (root
  .databases["GAIG_LAB"]
  .schemas["PUBLIC"]
  .cortex_search_services["parks_search_service"]
)

resp = parks_search_service.search(
  query="When will the Ault Park trail plan complete?",
  columns=["chunk"],
  limit=3
)

results = json.loads(resp.to_json())['results']

for idx, chunk in enumerate(results):
    print(f'Result: {idx+1}')
    print(chunk['chunk'])

## Part 4: Evals

In this section, we will:

1. Build out a sample of questions and ground truth results
2. Create a framework for running our samples through both standard search and Cortex search
3. Perform the evaluation and display the results

In [ ]:
eval_set = [
    {
        "query": "When will the Ault Park trail plan complete?",
        "expected_chunk": "AULT PARK VALLEY TRAIL One of the busiest trails in Cincinnati Parks is experiencing serious erosion issues along the creek also housing important sewer infrastructure. This project shores up the trail to keep hikers safe in advance of a larger MSD sewer project in the coming years to protect the trail in the long term. The project is underway and will close out in early 2025."
    },
    {
        "query": "When was the Sawyer Point Park playground burnt down?",
        "expected_chunk": "SAWYER POINT PLAYGROUND AND PARK PLANNING Work is underway to restore the Sawyer Point Park playground, which was suddenly destroyed by a massive fire in November 2024. This project represents a chance to create an amazing new, uniquely Cincinnati, amenity for the next generation of park users of a wide range of ages and abilities to enjoy. The new playground will be built in the vicinity of the former playground though not in the same location. The goal is to engage with the community to develop something truly fantastic in this iconic regional park serving as a distinctive source of lasting pride for our city. The project also creates an opportunity to comprehensively review the layout of the park to guide a longer-term plan for improvements in the coming years."
    },
    {
        "query": "How many miles is the CROWN network?",
        "expected_chunk": "BRAMBLE PARK TRAIL This project is a partnership with the community and will utilize a State of Ohio Department of Natural Resources grant. This will be the first segment of the Little Duck Creek Trail and run 0.35 miles in length through Bramble Park in Madisonville. The trail will connect to the Murray Trail, part of the CROWN network connecting more than 104 miles of trails in Cincinnati. Planning will take place during 2025."
    },
    {
        "query": "When did Smale park open?",
        "expected_chunk": "SMALE CONCRETE & GRANITE UPGRADES This award-winning, heavily used signature Cincinnati Park opened in 2012. Sections of concrete and specialized granite need repair in order to maintain this regional asset."
    },
    {
        "query": "Which park was added to the National Register of Historic Places?",
        "expected_chunk": "GIBSON HOUSE ROOF & FAÇADE This architecturally important structure was built in the middle 19th century and added to the National Register of Historic Places in 1976. Critical repairs are needed to preserve this treasure, which is now used for offices and a rental venue. Construction is planned to begin early 2026."
    },
    {
        "query": "Who is the Park Board partnering with for the Smale River's Edge project?",
        "expected_chunk": "SMALE RIVER’S EDGE The U.S. Army Corps of Engineers and the Cincinnati Park Board are partnering on a study to improve and revitalize the Cincinnati Ohio River’s edge along the western edge of Smale Riverfront Park. The overall vision is to make the Cincinnati Riverfront a welcoming, safe, sustainable park, serving as a gateway to connect people to their heritage, community, and the natural environment for generations to come. The project will provide opportunities for ecosystem restoration and recreation, while protecting Cincinnati’s Riverfront from erosion. Initial design selection of this multi-million project will be complete in mid-2025 with construction planned to start in 2027."
    },
    {
        "query": "How long will the California Woods Hydrological plan take?",
        "expected_chunk": "CALIFORNIA WOODS HYDROLOGICAL PLAN DESIGN This amazing preserve is experiencing significant erosion issues that threaten long-term public access to the park. A specialized firm has been selected to develop a plan for sustainable interventions to the stream flow to mitigate the on-going erosion issues in the most environmentally sustainable manner. Investigation and design work begins in the second quarter of 2025 and will take about a year to finalize."
    },
    {
        "query": "Which park plan is partering with the Cincinnati Off Road Alliance?",
        "expected_chunk": "MT. AIRY BIKE SKILLS COURSE This partnership with the Cincinnati Parks Foundation and the Cincinnati Off Road Alliance (CORA) will nearly double the existing mileage of mountain biking trails within Mt. Airy Forest. It will be the first beginner natural surface trail experience within the city. With input from the community, the project has been funded and a contractor selected. The project is anticipated to be complete in early 2026."
    },
    {
        "query": "How many acres is the Cincinnati Park system comprised of?",
        "expected_chunk": "CINCINNAT PARKS PARK IMPROVEMENT PROJECTS 3-YEAR PLAN Cincinnati Parks' 5,000 acres consist of 8 regional parks, 70 neighborhood parks, 34 preserves and natural areas, 5 parkways, 65 miles of hiking trails, 80,000 street trees on 1,000 miles of City streets, 6 nature centers, 18 scenic overlooks, 52 playgrounds, 500 landscaped gardens, and over 100 picnic areas. With all of this to care for, there are constant needs of all shapes and sizes. Whether it be a bad sidewalk, an aging playground, a leaking roof, or a park that could use a complete facelift, there’s plenty to do to keep our parks looking great and best serving our residents and users. This is why the Board of Park Commissioners approved a work plan, generated by Parks staff, outlining projects underway and planned over the next 3 years. This plan represents a roadmap of what Cincinnati Parks will be prioritizing in the coming years and creates transparency into improvement projects. This was developed after a careful evaluation based on a number of factors including safety, equity, efficiencies, long-term maintenance, available funding, and more. This plan represents current priorities, capacity, and needs, and is a living document that will be updated as circumstances evolve and schedules are adjusted."
    },
    {
        "query": "Which communities does the Burnet Woods dog park serve?",
        "expected_chunk": "BURNET WOODS DOG PARK This new community dog park will serve Clifton, Corryville, CUF, and the surrounding areas, further contributing to the attractiveness and quality of life. The project represents a partnership with a number of community supporters, partners, and donors, including the Cincinnati Parks Foundation and Clifton Pop-up-Pup-Party (PUPP). The new amenity is expected to be under construction in May 2025 and take about 3 months to complete."
    }
    
]

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
import json
import re

session = Session.builder.getOrCreate()
model = 'snowflake-arctic-embed-m-v1.5'

def escape_sql_string(s):
    return s.replace("'", "''")

def vector_search(query, k=3):
    safe_query = escape_sql_string(query)
    return session.sql(f"""
        SELECT chunk, VECTOR_COSINE_SIMILARITY(
            docs_chunks_table.embedding,
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('{model}', '{safe_query}')
        ) AS similarity
        FROM docs_chunks_table
        ORDER BY similarity DESC
        LIMIT {k}
    """).collect()

def cortex_search(query, limit=3):
    parks_search_service = (root.databases["GAIG_LAB"]
                                   .schemas["PUBLIC"]
                                   .cortex_search_services["parks_search_service"])
    resp = parks_search_service.search(
        query=query,
        columns=["chunk"],
        limit=limit
    )
    results = json.loads(resp.to_json())['results']
    return results


def normalize(text):
    # Lowercase, remove extra whitespace, and normalize newlines
    return re.sub(r'\s+', ' ', text.strip().lower())


In [ ]:
def run_eval():
    results = []
    
    for item in eval_set:
        query = item['query']
        expected = item['expected_chunk']
    
        # Vector search
        vector_results = vector_search(query, k=3)
        vector_chunks = [r['CHUNK'] for r in vector_results]
        vector_match_rank = next(
            (i + 1 for i, chunk in enumerate(vector_chunks)
             if normalize(chunk) == normalize(expected)),
            None
        )
    
        # Cortex search
        cortex_results = cortex_search(query, limit=3)
        cortex_chunks = [r['chunk'] for r in cortex_results]
        cortex_match_rank = next(
            (i + 1 for i, chunk in enumerate(cortex_chunks)
             if normalize(chunk) == normalize(expected)),
            None
        )
    
        results.append({
            "query": query,
            "expected_chunk": expected,
            "vector_hit_in_top3": vector_match_rank is not None,
            "vector_hit_rank": vector_match_rank or "Miss",
            "vector_chunks": vector_chunks,
            "cortex_hit_in_top3": cortex_match_rank is not None,
            "cortex_hit_rank": cortex_match_rank or "Miss",
            "cortex_chunks": cortex_chunks
        })
    return results

results = run_eval()

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df[['query', 'vector_hit_in_top3', 'cortex_hit_in_top3']]